In [1]:
import csv
import random
import json
from collections import defaultdict

## Preprocess data

In [2]:
filename = './archive/RAW_recipes.csv'
reader = csv.DictReader(open(filename))

recipesData = list()
for rows in reader:
    recipe = (rows['id'], rows)
    recipesData.append(recipe)

In [3]:
filename = './archive/RAW_interactions.csv'
reader = csv.DictReader(open(filename))

reviewsData = list()
for rows in reader:
    review = (rows['user_id'], rows['recipe_id'], rows)
    reviewsData.append(review)

In [4]:
users = set([r[0] for r in reviewsData])

In [5]:
print('Number of recipes: ' + str(len(recipesData)))
print('Number of reviews: ' + str(len(reviewsData)))
print('Number of users: ' + str(len(users)))

Number of recipes: 231637
Number of reviews: 1132367
Number of users: 226570


## Split dataset

In [6]:
random.seed(0)
random.shuffle(reviewsData)

In [7]:
# Test code with 5% of dataset
#reviewsData = reviewsData[:round(len(reviewsData)*.05)]

In [8]:
# 60:20:20
reviewsTrain = reviewsData[:round(len(reviewsData)*.6)]
reviewsValid = reviewsData[round(len(reviewsData)*.6):round(len(reviewsData)*.8)]
reviewsTest = reviewsData[round(len(reviewsData)*.8):]

In [9]:
len(reviewsValid)

226474

## Cook prediction task

In [15]:
usersPerRecipe = defaultdict(set) # Maps a recipe to the users who rated it
RecipesPerUser = defaultdict(set) # Maps a user to the recipes they rated
allRecipes = list()

for user,recipe,_ in reviewsData:
    
    usersPerRecipe[recipe].add(user)
    RecipesPerUser[user].add(recipe)
    
    allRecipes.append(recipe)

allRecipes = set(allRecipes)

for _,_,meta in reviewsValid:
    meta['cooked'] = 1

In [12]:
# Sample negative entry in validation set

#negValidData = list(reviewsValid)

#for user,recipe,meta in reviewsValid:
#    negRecipe = random.sample(sorted(allRecipes - RecipesPerUser[user]), 1)[0]
#    negEntry = (str(user), str(negRecipe), {'user_id': user, 
#                                  'recipe_id': negRecipe,
#                                  'cooked': 0} )
#    negValidData.append(negEntry)
    
#with open('negValidData.json', 'w') as f:
#    json.dump(tuple(negValidData), f)

In [14]:
# Read json with negative entries from validation set

f = open("negValidData.json")

negValidData = []

for l in f:
    negValidData.append(eval(l))
    
negValidData = negValidData[0]

### Baseline 1 (based on recipe popularity)

In [16]:
recipeCount = defaultdict(int)
totalCooked = 0

for user,recipe,_ in reviewsTrain:
    recipeCount[recipe] += 1
    totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

returnPred = set()
count = 0
for ic, i in mostPopular:
    count += ic
    returnPred.add(i)
    if count > totalCooked/2: break

In [17]:
# Evaluate Baseline 1 on validation data

validPred = []

for user,recipe,meta in negValidData:
    
    recipeCookedFlag = meta['cooked']
    
    if recipe in returnPred:
        if recipeCookedFlag:
            validPred.append(1)
        else:
            validPred.append(0)
    else:
        if not recipeCookedFlag:
            validPred.append(1)
        else:
            validPred.append(0)

In [18]:
accuracy = sum(validPred)/len(negValidData)

print("Accuracy on validation set = " + str(accuracy))

Accuracy on validation set = 0.6970071619700274


### Baseline 2 (based on recipe similarity)

In [19]:
usersPerRecipe = defaultdict(set) # Maps a recipe to the users who rated it
RecipesPerUser = defaultdict(set) # Maps a user to the recipes they rated
recipeCount = defaultdict(int)
totalCooked = 0

for user,recipe,_ in reviewsTrain:
#for user,recipe,_ in reviewsData:
    
    recipeCount[recipe] += 1
    totalCooked += 1
    
    usersPerRecipe[recipe].add(user)
    RecipesPerUser[user].add(recipe)

In [20]:
def Jaccard(s1, s2):
    
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))

    if denom == 0:
        print("denom = 0")
    
    return numer/denom

In [23]:
thresholds = [1/2**i for i in range(1, 10)]
bestAccuracy = 0
bestThres = 0

#for t in thresholds:
for t in [0.001]:
    print('t = ' + str(t))
    
    validPred = []
    
    for user,recipe,meta in negValidData:
        recipesCooked = RecipesPerUser[user]
        
        #if len(recipesCooked) == 0: continue
        
        similarities = []
        for recipePrime in recipesCooked:
            
            sim = Jaccard(usersPerRecipe[recipe], usersPerRecipe[recipePrime])
            similarities.append(sim)
        
        recipeCookedFlag = meta['cooked'] 
        if len(similarities) > 0 and max(similarities) > t:
            if recipeCookedFlag:
                validPred.append(1)
            else:
                validPred.append(0)
        else:
            if not recipeCookedFlag:
                validPred.append(1)
            else:
                validPred.append(0)
    
    accuracy = sum(validPred)/len(negValidData)
    print(accuracy)
    
    if bestAccuracy < accuracy:
        bestAccuracy = accuracy
        bestThres = t

t = 0.001
0.6136775082349408


In [34]:
#print("\nAccuracy = " + str(bestAccuracy))
#print("Threshold = " + str(bestThres))

### Baseline 3 (based on recipe popularity and similariy)

In [31]:
bestAccuracy = 0
bestThres1 = 0
bestThres2 = 0

#for x in range(100,120)
#    for y in range(1,9):
for x in [119]:
    for y in [1]:
        thre1=x*0.005
        thre2=y*0.01
        
        thresholdPopu = totalCooked*thre1
        thresholdSim = thre2

        print('Threshold 1 = ' + str(thre1))
        print('Threshold 2 = ' + str(thresholdSim))

        # Train
        mostPopular = [(recipeCount[x], x) for x in recipeCount]
        mostPopular.sort()
        mostPopular.reverse()

        returnPred = set()
        count = 0
        for ic, i in mostPopular:
            count += ic
            returnPred.add(i)
            if count > thresholdPopu: break

        # Evaluate
        validPred = []

        for user,recipe,meta in negValidData:
            recipesCooked = RecipesPerUser[user]

            similarities = []
            for recipePrime in recipesCooked:
                sim = Jaccard(usersPerRecipe[recipe], usersPerRecipe[recipePrime])
                similarities.append(sim)

            recipeCookedFlag = meta['cooked'] 
            if len(similarities) > 0 and max(similarities) > thresholdSim and recipe in returnPred:
                if recipeCookedFlag:
                    validPred.append(1)
                else:
                    validPred.append(0)
            else:
                if not recipeCookedFlag:
                    validPred.append(1)
                else:
                    validPred.append(0)

        accuracy = sum(validPred)/len(negValidData)
        print("Accuracy = " + str(accuracy))
        
        if bestAccuracy < accuracy:
            bestAccuracy = accuracy
            bestThres1 = thresholdPopu
            bestThres2 = thresholdSim

Threshold 1 = 0.595
Threshold 2 = 0.01
Accuracy = 0.6246213693404099


In [33]:
#print("\nAccuracy = " + str(bestAccuracy))
#print("Threshold 1 = " + str(bestThres1))
#print("Threshold 2 = " + str(bestThres2))